## What is Numba?

a JIT (Just-in-Time) compiler for Python that:



- generates optimized machine code using LLVM (Low Level Virtual Machine) compiler infrastructure


- provides toolbox for different targets and execution models:
    - Single-threaded CPU, multi-threaded CPU, GPU
    - regular functions, "universal functions (ufuncs)" (array functions), etc


- integrates well with the Scientific Python stack


- with a few annotations, array-oriented and math-heavy Python code provides: 
 - speedup: 2x (compared to basic NumPy code) to 200x (compared to pure Python)
  - performance similar to C, C++, Fortran, without having to switch languages or Python interpreters


- is **totally awesome!**

## Basic Example

### Lazy Compilation

- Use `@jit` decorator
- Let Numba decide when and how to optimize

In [1]:
import numpy as np
import numba
from numba import jit

In [2]:
@jit
def do_math(x, y):
    return x + y

In this mode:

- The compilation will be deferred until the first execution
- Numba will:
    - infer the argument types at call time
    - generate optimized code based on this information
- Numba will also be able to compile separate specializations depending on the input types. For instance, calling `do_math()` with integer or complex numbers will generate different code paths:

In [3]:
do_math.inspect_types()

In [4]:
%time do_math(1, 2)

CPU times: user 114 ms, sys: 31.8 ms, total: 146 ms
Wall time: 762 ms


3

In [5]:
%time do_math(1, 2)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.34 µs


3


**What is Numba doing to make code run quickly?**

Numba examines Python bytecode and then translates this into an 'intermediate representation'.  To view this IR, after running (compiling) `do_math()` and you can access the `inspect_types` method.

In [6]:
do_math.inspect_types()

do_math (int64, int64)
--------------------------------------------------------------------------------
# File: <ipython-input-2-10b52aae7618>
# --- LINE 1 --- 
# label 0

@jit

# --- LINE 2 --- 

def do_math(x, y):

    # --- LINE 3 --- 
    #   x = arg(0, name=x)  :: int64
    #   y = arg(1, name=y)  :: int64
    #   $0.3 = x + y  :: int64
    #   del y
    #   del x
    #   $0.4 = cast(value=$0.3)  :: int64
    #   del $0.3
    #   return $0.4

    return x + y




In [7]:
%time do_math(1j, 2)

CPU times: user 35.1 ms, sys: 1.95 ms, total: 37.1 ms
Wall time: 35.5 ms


(2+1j)

In [8]:
%time do_math(1j, 2)

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 7.87 µs


(2+1j)

In [9]:
do_math.inspect_types()

do_math (int64, int64)
--------------------------------------------------------------------------------
# File: <ipython-input-2-10b52aae7618>
# --- LINE 1 --- 
# label 0

@jit

# --- LINE 2 --- 

def do_math(x, y):

    # --- LINE 3 --- 
    #   x = arg(0, name=x)  :: int64
    #   y = arg(1, name=y)  :: int64
    #   $0.3 = x + y  :: int64
    #   del y
    #   del x
    #   $0.4 = cast(value=$0.3)  :: int64
    #   del $0.3
    #   return $0.4

    return x + y


do_math (complex128, int64)
--------------------------------------------------------------------------------
# File: <ipython-input-2-10b52aae7618>
# --- LINE 1 --- 
# label 0

@jit

# --- LINE 2 --- 

def do_math(x, y):

    # --- LINE 3 --- 
    #   x = arg(0, name=x)  :: complex128
    #   y = arg(1, name=y)  :: int64
    #   $0.3 = x + y  :: complex128
    #   del y
    #   del x
    #   $0.4 = cast(value=$0.3)  :: complex128
    #   del $0.3
    #   return $0.4

    return x + y




### Eager compilation

- Tell Numba the function signature you are expecting

In [10]:
from numba import int32

In [11]:
@jit(int32(int32, int32))
def eager_do_math(x, y):
    return x + y

In [12]:
%time eager_do_math(1, 2)

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 9.54 µs


3

In [13]:
%time eager_do_math(1.0, 2.0)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 8.11 µs


3

In [14]:
%time eager_do_math(1j, 2)

TypeError: No matching definition for argument type(s) complex128, int64

## How does Numba work?

![](./images/how-does-numba-work.png)

Source: [Scaling Python Up and Out with Numba and Dask — Travis Oliphant](https://speakerdeck.com/teoliphant/scaling-python-up-and-out-with-numba-and-dask?slide=37)



### What about the actual LLVM code?
You can see the actual LLVM code generated by Numba using the `inspect_llvm()` method. 

In [15]:
for key, value in do_math.inspect_llvm().items():
    print(key, value)

(int64, int64) ; ModuleID = 'do_math'
source_filename = "<string>"
target datalayout = "e-m:e-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-unknown-linux-gnu"

@"_ZN08NumbaEnv8__main__11do_math$241Exx" = common local_unnamed_addr global i8* null
@.const.do_math = internal constant [8 x i8] c"do_math\00"
@PyExc_RuntimeError = external global i8
@".const.missing Environment" = internal constant [20 x i8] c"missing Environment\00"

; Function Attrs: norecurse nounwind
define i32 @"_ZN8__main__11do_math$241Exx"(i64* noalias nocapture %retptr, { i8*, i32 }** noalias nocapture readnone %excinfo, i64 %arg.x, i64 %arg.y) local_unnamed_addr #0 {
entry:
  %.14 = add nsw i64 %arg.y, %arg.x
  store i64 %.14, i64* %retptr, align 8
  ret i32 0
}

define i8* @"_ZN7cpython8__main__11do_math$241Exx"(i8* nocapture readnone %py_closure, i8* %py_args, i8* nocapture readnone %py_kws) local_unnamed_addr {
entry:
  %.5 = alloca i8*, align 8
  %.6 = alloca i8*, align 8
  %.7 = call i32 (i8*, i8*, i

**But there's a caveat....**

### Compilation Options

Numba has two compilation modes:

- **nopython mode (recommended and best-practice way)**: produces much faster code by running the code without the involvement of the Python interpreter.

- **object mode (should be avoided)**: Numba falls back to this mode when `nopython` mode fails.

To illustrate the above, let's watch what happens when we try to do something that is natural in Python (concatenating strings), but not particularly mathematically sound:

In [16]:
%time do_math('Hello', 'World')

CPU times: user 543 ms, sys: 1.56 ms, total: 544 ms
Wall time: 563 ms


'HelloWorld'

In [17]:
do_math.inspect_types()

do_math (int64, int64)
--------------------------------------------------------------------------------
# File: <ipython-input-2-10b52aae7618>
# --- LINE 1 --- 
# label 0

@jit

# --- LINE 2 --- 

def do_math(x, y):

    # --- LINE 3 --- 
    #   x = arg(0, name=x)  :: int64
    #   y = arg(1, name=y)  :: int64
    #   $0.3 = x + y  :: int64
    #   del y
    #   del x
    #   $0.4 = cast(value=$0.3)  :: int64
    #   del $0.3
    #   return $0.4

    return x + y


do_math (complex128, int64)
--------------------------------------------------------------------------------
# File: <ipython-input-2-10b52aae7618>
# --- LINE 1 --- 
# label 0

@jit

# --- LINE 2 --- 

def do_math(x, y):

    # --- LINE 3 --- 
    #   x = arg(0, name=x)  :: complex128
    #   y = arg(1, name=y)  :: int64
    #   $0.3 = x + y  :: complex128
    #   del y
    #   del x
    #   $0.4 = cast(value=$0.3)  :: complex128
    #   del $0.3
    #   return $0.4

    return x + y


do_math (unicode_type, unicode_type)
-

`do_math (unicode_type, unicode_type)` means that is has been compiled in `object` mode. 

To prevent Numba from falling back, and instead raise an error, we need to pass `nopython=True` to `@jit` decorator:

In [18]:
@jit
def f(x, y): # Function will not befenit from Numba jit
    a = str(x) * 10 # Numba doesn't know about str
    b = str(y)
    return a + b 

In [19]:
%timeit f(1, 2)

3.31 µs ± 17.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [20]:
@jit(nopython=True) # Fore nopython mode
def f(x, y): # Function will not befenit from Numba jit
    a = str(x) * 10 # Numba doesn't know about str
    b = str(y)
    return a + b 

In [21]:
%timeit f(1, 2)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'str': cannot determine Numba type of <class 'type'>

File "<ipython-input-20-d592c0643be0>", line 3:
def f(x, y): # Function will not befenit from Numba jit
    a = str(x) * 10 # Numba doesn't know about str
    ^

This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To see Python/NumPy features supported by the latest release of Numba visit:
http://numba.pydata.org/numba-doc/dev/reference/pysupported.html
and
http://numba.pydata.org/numba-doc/dev/reference/numpysupported.html

For more information about typing errors and how to debug them visit:
http://numba.pydata.org/numba-doc/latest/user/troubleshoot.html#my-code-doesn-t-compile

If you think your code should work with Numba, please report the error message
and traceback, along with a minimal reproducer at:
https://github.com/numba/numba/issues/new


## Benchmarks using the all pairwise distance function

### Pure Python Version

In [25]:
def allpairs_distances_python(X,Y):
    result = np.zeros( (X.shape[0], Y.shape[0]), X.dtype)
    for i in range(X.shape[0]):
        for j in range(Y.shape[0]):
            result[i,j] = np.sum( (X[i,:] - Y[j,:]) ** 2)
    return result 

In [26]:
N = 1000 
X, Y = np.random.randn(200, N), np.random.randn(400, N)
X.shape, Y.shape 

((200, 1000), (400, 1000))

In [27]:
pure_python = %timeit -o allpairs_distances_python(X, Y)

1.32 s ± 6.88 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Where is the bottleneck?

In [28]:
%load_ext line_profiler

In [29]:
%lprun -f allpairs_distances_python allpairs_distances_python(X,Y)

### Numba Version

In [30]:
from numba import jit

@jit(nopython=True)
def allpairs_distances_numba(X,Y):
    result = np.zeros((X.shape[0], Y.shape[0]), X.dtype)
    for i in range(X.shape[0]):
        for j in range(Y.shape[0]):
            result[i,j] = np.sum( (X[i,:] - Y[j,:]) ** 2)
    return result 

I should emphasize that this is the exact same code, except for numba's `jit` decorator. The results are pretty astonishing:

In [31]:
numba_version = %timeit -o allpairs_distances_numba(X,Y)

124 ms ± 464 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
speedup = pure_python.best / numba_version.best 

In [33]:
print(f"This is a {round(speedup, 0)}x speedup, simply by adding a Numba decorator!")

This is a 11.0x speedup, simply by adding a Numba decorator!


## Loops


While NumPy has developed a strong idiom  around the use of **vectorized** operations, Numba is perfectly happy with loops too.

For C, and Fortran users, writing Python in this style will work fine in Numba (Thanks to LLVM!)

In [34]:
from numba import njit # njit is an alias for @jit(nopython=True)

In [35]:
# Pure NumPy
def ident_numpy(x):
    return np.cos(x) ** 2 + np.sin(x) ** 2

# Jitted NumPy 
@njit
def ident_numpy_jit(x):
    return np.cos(x) ** 2 + np.sin(x) ** 2

# NumPy with loops
def ident_numpy_loops(x):
    r = np.empty_like(x)
    n = len(x)
    for i in range(n):
        r[i] = np.cos(x[i] ** 2 + np.sin(x[1]) ** 2)
        
    return r 

# Jitted NumPy with loops 
@njit
def ident_numpy_loops_jit(x):
    r = np.empty_like(x)
    n = len(x)
    for i in range(n):
        r[i] = np.cos(x[i] ** 2 + np.sin(x[1]) ** 2)
        
    return r 


In [36]:
x = np.arange(1.e6)

In [37]:
%timeit ident_numpy(x)

77.3 ms ± 150 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [38]:
%timeit ident_numpy_jit(x)

78 ms ± 2.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [39]:
%timeit ident_numpy_loops(x)

6.41 s ± 5.61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
%timeit ident_numpy_loops_jit(x)

49.3 ms ± 391 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Creating  NumPy Universal Functions (Ufuncs)

- Ufuncs are a core concept in NumPy for array-oriented computig.

  - A function with scalar inputs is broadcast across the elements of the input arrays:
      

In [41]:
np.add([1, 2, 3], 3)

array([4, 5, 6])

In [42]:
np.add([1, 2, 3], [10, 20, 30])

array([11, 22, 33])

- Before Numba, creating fast ufuncs required writing C. **This is no longer the case!**


There's a tutorial on how to write ufuncs in NumPy from  [documentation](http://docs.scipy.org/doc/numpy/user/c-info.ufunc-tutorial.html), the example they post there is a ufunc to perform 

$$f(a) = \log \left(\frac{a}{1-a}\right)$$

It looks like this:

```c
static void double_logit(char **args, npy_intp *dimensions,
                            npy_intp* steps, void* data)
{
    npy_intp i;
    npy_intp n = dimensions[0];
    char *in = args[0], *out = args[1];
    npy_intp in_step = steps[0], out_step = steps[1];

    double tmp;

    for (i = 0; i < n; i++) {
        /*BEGIN main ufunc computation*/
        tmp = *(double *)in;
        tmp /= 1-tmp;
        *((double *)out) = log(tmp);
        /*END main ufunc computation*/

        in += in_step;
        out += out_step;
    }
}
```

**NOTE:** That's just for a `double`.  If you want `floats`, `long doubles`, etc... you have to write all of those, too.  And then create a `setup.py` file to install it, etc.

### The `@vectorize` decorator

In [43]:
import math

In [44]:
def trig_func(x, y):
    return ((math.sin(x) ** 2) + (math.cos(y) ** 2))

In [45]:
trig_func(1, 1.5)

0.7130771699733485

Seems reasonable.  However, the `math` library only works on scalars.  If we try to pass in arrays, we'll get an error.

In [46]:
trig_func([1, 2], [1, 2])

TypeError: must be real number, not list

Using `@vectorize` decorator, we are able to write our function as operating over input scalars, rather than arrays. Numba will generate teh surrounding loop (or kernel) allowing efficient iteration over the actual inputs.

In [47]:
from numba import vectorize, float64, float32, int32, int64

In [48]:
# Define ufunc with multiple signatures
@vectorize(['int32(int32, int32)',
            'int64(int64, int64)',
            'float32(float32, float32)',
            'float64(float64, float64)'])
def trig_func(x, y):
    return ((math.sin(x) ** 2) + (math.cos(y) ** 2))

And just like that, the scalar function `trig_func` is now a NumPy `ufunc` called `trig_func`

In [49]:
a = np.random.random((1000, 1000))
b = np.random.random((1000, 1000))

In [50]:
%timeit trig_func(a, b)

37.4 ms ± 124 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [51]:
def trig_func_numpy(x, y):
    return ((np.sin(x) ** 2) + (np.cos(y) ** 2))

How does it compare to just using NumPy?  Let's check

In [52]:
%timeit trig_func_numpy(a, b)

38.9 ms ± 166 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


**NOTE**: NumPy ufuncs automatically get other features such as:

- reduction
- accumulation
- broadcasting

By defining our ufunc using Numba, we get these additional features for **free**.

In [53]:
a = np.arange(12).reshape(3, 4)
a

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [54]:
trig_func.reduce(a, axis=0)

array([0, 0, 1, 0])

In [55]:
trig_func.accumulate(a)

array([[0, 1, 2, 3],
       [0, 0, 1, 0],
       [0, 0, 1, 0]])

In [ ]:
%load_ext watermark

In [ ]:
%watermark -u -n -t -iv -g -m


**To Be Continued...................**

<h1 align="center">The Sheer Joy of Accelerating Your Existing Python Code with Numba!</h1>

<h2 align="center">Part II : Numba for Cuda GPUs</h2>
